## 영상의 필터링

* 엠보싱 필터

In [3]:
import cv2
import numpy as np

In [9]:
img = cv2.imread('./data/rose.bmp')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

mask = np.array([[-1, -1, 0], [-1, 0, 1], [0, 1, 1]])
out = cv2.filter2D(gray, -1, mask) # -1 : src와 output 채널 같게 하겠다.

cv2.imshow('img', img)
cv2.imshow('out', out)

cv2.waitKey()
cv2.destroyAllWindows()

In [6]:
img.shape

(320, 480, 3)

* 평균값 필터(blur, boxFilter)

In [10]:
src = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)

dst = cv2.blur(src, ksize=(5, 5))

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [11]:
src = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)

dst = cv2.boxFilter(src, -1, ksize=(5, 5))

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

* 가우시안 필터

In [14]:
src = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)
dst = cv2.GaussianBlur(src, ksize=(7, 7), sigmaX=0, sigmaY=0)

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [17]:
img = cv2.imread('./data/rose.bmp', cv2.IMREAD_GRAYSCALE)

for i in range(3):
    dst = cv2.GaussianBlur(img, ksize=(7, 7), sigmaX=(i+1))
    cv2.imshow(f'sigmaX = {i+1}', dst)
cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()

* 언샤프 마스크 필터

In [2]:
img = cv2.imread('./data/rose.bmp', cv2.IMREAD_GRAYSCALE)

for i in range(3):
    blurred = cv2.GaussianBlur(img, ksize=(7, 7), sigmaX=(i+1))
    alpha = 1.0
    dst = (1+alpha)*img - (alpha*blurred)
    dst = np.uint8(dst)
    cv2.imshow(f'sigmaX={i+1}', dst)
    
cv2.waitKey()
cv2.destroyAllWindows()

* 양방향 필터

In [9]:
src = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)

data = np.zeros(src.shape, dtype=np.uint8)
noise = cv2.randn(data, mean=0, stddev=10)

src_noise = cv2.add(src, noise)

dst1 = cv2.GaussianBlur(src_noise, ksize=(7, 7), sigmaX=10)
dst2 = cv2.bilateralFilter(src_noise, -1, sigmaColor=10, sigmaSpace=10)

cv2.imshow('src', src)
cv2.imshow('src_noise', src_noise)
cv2.imshow('dst1', dst1)
cv2.imshow('dst2', dst2)

cv2.waitKey()
cv2.destroyAllWindows()

* 미디언 필터

In [18]:
src = cv2.imread('./data/Lena.bmp', cv2.IMREAD_GRAYSCALE)

num = src.shape[0] * src.shape[1] * 0.1

for i in range(np.int64(num)):
    x = np.random.randint(src.shape[1])
    y = np.random.randint(src.shape[0])
    
    src[y, x] = (i % 2) * 255

gaussian = cv2.GaussianBlur(src, ksize=(7, 7), sigmaX=5)
median = cv2.medianBlur(src, ksize=7)
    
cv2.imshow('src', src)
cv2.imshow('gaussian', gaussian)
cv2.imshow('median', median)

cv2.waitKey()
cv2.destroyAllWindows()

## 모폴로지 연산

In [27]:
src = cv2.imread('./data/morphology.jpg', cv2.IMREAD_GRAYSCALE)

kernel = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(3, 3))
erode = cv2.erode(src, kernel, iterations=5)
dilate = cv2.dilate(src, kernel, iterations=5)

cv2.imshow('src', src)
cv2.imshow('erode', erode)
cv2.imshow('dilate', dilate)
cv2.waitKey()
cv2.destroyAllWindows()

* 열기와 닫기

In [32]:
src = cv2.imread('./data/morphology.jpg', cv2.IMREAD_GRAYSCALE)

kernel = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(3, 3))
closing = cv2.morphologyEx(src, cv2.MORPH_CLOSE, kernel, iterations=4)
opening = cv2.morphologyEx(src, cv2.MORPH_OPEN, kernel, iterations=4)
open_close = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=4)

cv2.imshow('src', src)
cv2.imshow('closing', closing)
cv2.imshow('opening', opening)
cv2.imshow('open_close', open_close)
cv2.waitKey()
cv2.destroyAllWindows()

## 에지 검출과 응용

* 미분 필터

In [14]:
src = cv2.imread('./data/rect.jpg', cv2.IMREAD_GRAYSCALE)
# src = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)

gx = cv2.Sobel(src, cv2.CV_32F, 1, 0, ksize=3) # 1, 0 : x축, y축 (x축 방향으로 미분)
gy = cv2.Sobel(src, cv2.CV_32F, 0, 1, ksize=3) # 0, 1 : x축, y축 (y축 방향으로 미분)

dstX = cv2.sqrt(np.abs(gx))
dstX = cv2.normalize(dstX, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

dstY = cv2.sqrt(np.abs(gy))
dstY = cv2.normalize(dstY, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

mag = cv2.magnitude(gx, gy)
dstM = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U) # dstX, dstY 합친 것

ret, edge = cv2.threshold(mag, 100, 255, cv2.THRESH_BINARY)
edge = edge.astype(np.uint8)

cv2.imshow('src', src)
cv2.imshow('dstX', dstX)
cv2.imshow('dstY', dstY)
cv2.imshow('dstM', dstM)
cv2.imshow('edge', edge)
cv2.waitKey()
cv2.destroyAllWindows()

In [23]:
src = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)

blur = cv2.GaussianBlur(src, ksize=(7, 7), sigmaX=0)

gx1 = cv2.Sobel(src, cv2.CV_32F, 1, 0, ksize=3)
gy1 = cv2.Sobel(src, cv2.CV_32F, 0, 1, ksize=3)
mag1, angle1 = cv2.cartToPolar(gx1, gy1, angleInDegrees=True) # False : radians(라디안 방식)

gx2 = cv2.Sobel(blur, cv2.CV_32F, 1, 0, ksize=3)
gy2 = cv2.Sobel(blur, cv2.CV_32F, 0, 1, ksize=3)
mag2, angle2 = cv2.cartToPolar(gx2, gy2, angleInDegrees=True)

ret1, edge1 = cv2.threshold(mag1, 100, 255, cv2.THRESH_BINARY)
edge1 = edge1.astype(np.uint8)

ret2, edge2 = cv2.threshold(mag2, 100, 255, cv2.THRESH_BINARY)
edge2 = edge2.astype(np.uint8)

cv2.imshow('edge1', edge1)
cv2.imshow('edge2', edge2)
cv2.waitKey()
cv2.destroyAllWindows()